In [1]:
# import hilbert
import conversions

#includes for genetic
import population
import psnr
import crossover
import embed
import decode
import selection


In [2]:
import cv2

# image_name = input("Enter Cover Image Name: ")
image_name = "lena_gray.bmp"
image = cv2.imread(image_name, 0)


secretData = open("secret.txt",'r').read()

it = 4

mainPopulation = population.genPopulation(4)

print(mainPopulation)
fitnessArray = []*it
selectionArray ={}


for i in range(it):
    print("Parent - ", i)
    image = cv2.imread(image_name, 0)
    stegoImage = embed.encodeFunction(mainPopulation[i],image,secretData)
    originalImage = cv2.imread(image_name, 0)
    print(psnr.computeFitness(512, originalImage, stegoImage))
    fitnessArray.append(psnr.computeFitness(512, originalImage, stegoImage))

print(fitnessArray)

selection.selectionFunction(fitnessArray, mainPopulation,selectionArray)

for i in range(it):
    print("Child - ", i)

    newPop = crossover.crossoverFunction(mainPopulation, it)
    mainPopulation = newPop
    fitnessArray = []*it
    print(newPop)


    for j in range(it):
        print("Childs - ", i, j)
        image = cv2.imread(image_name, 0)
        print(newPop[j])
        stegoImage = embed.encodeFunction(newPop[j],image,secretData)

        originalImage = cv2.imread(image_name, 0)
        print(psnr.computeFitness(512, originalImage, stegoImage))

        fitnessArray.append(psnr.computeFitness(512, originalImage, stegoImage))

    print(fitnessArray)

    selection.selectionFunction(fitnessArray, newPop,selectionArray)


print(selectionArray)

finalBestChrom = ''
for x in sorted(selectionArray.keys(), reverse=True):
    finalBestChromValue = x
    finalBestChrom = selectionArray[x]
    break
        
selectionFile = open("SelectionArray.txt", 'w')

for x in sorted(selectionArray.keys(), reverse=True):
    selectionFile.write(str(selectionArray[x]) + '->'+str(x))

print("Final Best Chromosome using PSRN is: ", finalBestChrom,
        "and the fitness value is: ", finalBestChromValue)

stegoImage = embed.encodeFunction(finalBestChrom,image,secretData)

cv2.imwrite("encoded_image.bmp",stegoImage)


# decode.decodeFunction("00000000000000000000",stegoImage)

# print(fitnessArray)



['10010010110010101101', '10011100111001011111', '11101101101011101100', '11110000100100010000']
Parent -  0


d:\Project\7th Sem Project\genetic-hilbert-embedding\psnr.py:9: RuntimeWarning: overflow encountered in ubyte_scalars
  dataCng = originalImage[i][j] - stegoImage[i][j]
d:\Project\7th Sem Project\genetic-hilbert-embedding\psnr.py:10: RuntimeWarning: overflow encountered in ubyte_scalars
  dataCng = dataCng * dataCng


Computed Fitness
41.04591330561494
Computed Fitness
Parent -  1
Computed Fitness
41.09290355764796
Computed Fitness
Parent -  2
Computed Fitness
41.04960377945969
Computed Fitness
Parent -  3
Computed Fitness
41.018050818798535
Computed Fitness
[41.04591330561494, 41.09290355764796, 41.04960377945969, 41.018050818798535]
Child -  0
['10010010110010101001110', '10011100111001011001001', '11101101101011101111000', '11110000100100011110110']
Childs -  0 0
10010010110010101001110
Computed Fitness
41.055567698297295
Computed Fitness
Childs -  0 1
10011100111001011001001
Computed Fitness
41.102804318345044
Computed Fitness
Childs -  0 2
11101101101011101111000
Computed Fitness
41.04175636182865
Computed Fitness
Childs -  0 3
11110000100100011110110
Computed Fitness
41.04304222412705
Computed Fitness
[41.055567698297295, 41.102804318345044, 41.04175636182865, 41.04304222412705]
Child -  1
['10010010110010101001110', '10011100111001011001001', '11101101101011101111000', '1111000010010001111011